In [10]:
import os, glob


import random
import sys
import pickle

import pandas as pd

from gensim.corpora.dictionary import Dictionary

# LSTM and CNN for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint

from nltk import word_tokenize

In [2]:
train_dfs = []
traindir = 'CWI 2018 Training Set/'
for lang in os.listdir(traindir):
    for filename in glob.iglob(traindir+lang+'/*.tsv', recursive=True):
        df = pd.read_csv(filename, header=None, sep='\t')
        df = df.rename(columns={0:'HIT_id', 1:'sentence', 2:'start', 3:'end',
                               4:'target_word', 5:'native_all', 6:'non_native_all', 
                               7:'native_score', 8:'non_native_score',
                               9:'binary_label', 10:'prob_label'})
        df['filename'] = filename
        df['lang'] = lang
        train_dfs.append(df)
        
train_df = pd.concat(train_dfs)

In [3]:
train_df['target_word_lower'] = train_df['target_word'].astype(str).apply(str.lower)
train_df['sentence_lower'] = train_df['sentence'].astype(str).apply(str.lower).apply(word_tokenize).apply(lambda x: ' '.join(x))

train_df['doc'] =  train_df['target_word_lower'] + ' <s> '  + train_df['sentence_lower']
train_df['lang_doc'] =  train_df['lang'] + ' <l> ' + train_df['doc']

train_doc = train_df['lang_doc'].apply(str.split)

In [4]:
test_dfs = {}
testdir = 'CWI 2018 Test Set/'
for lang in os.listdir(testdir):
    for filename in glob.iglob(testdir+lang+'/*.tsv', recursive=True):
        df = pd.read_csv(filename, header=None, sep='\t')
        df = df.rename(columns={0:'HIT_id', 1:'sentence', 2:'start', 3:'end',
                               4:'target_word', 5:'native_all', 6:'non_native_all', 
                               7:'native_score', 8:'non_native_score',
                               9:'binary_label', 10:'prob_label'})
        df['filename'] = filename
        df['lang'] = lang
        test_dfs[lang] = df

In [5]:
test_docs = {}
for lang in test_dfs.keys():
    test_dfs[lang]['target_word_lower'] = test_dfs[lang]['target_word'].astype(str).apply(str.lower)
    test_dfs[lang]['sentence_lower'] = test_dfs[lang]['sentence'].astype(str).apply(str.lower).apply(word_tokenize).apply(lambda x: ' '.join(x))
    test_dfs[lang]['doc'] =  test_dfs[lang]['target_word_lower'] + ' <s> '  + test_dfs[lang]['sentence_lower']
    test_dfs[lang]['lang_doc'] =  test_dfs[lang]['lang'] + ' <l> ' + test_dfs[lang]['doc']
    test_docs[lang] = test_dfs[lang]['lang_doc'].apply(str.split)

In [6]:
vocab = Dictionary(train_doc)

def vectorize_sent(sent):
    return vocab.doc2idx(sent)

X_train = sequence.pad_sequences(train_doc.apply(vectorize_sent), maxlen=150)
y_train = train_df['binary_label']

X_test_en= sequence.pad_sequences(test_docs['english'].apply(vectorize_sent), maxlen=150)
X_test_es = sequence.pad_sequences(test_docs['spanish'].apply(vectorize_sent), maxlen=150)
X_test_de = sequence.pad_sequences(test_docs['german'].apply(vectorize_sent), maxlen=150)


In [8]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(len(vocab.keys()), 32, input_length=150))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 32)           657984    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 150, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 75, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 714,389
Trainable params: 714,389
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
filepath="{epoch:02d}-{acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the mode

In [ ]:
model.fit(X_train, y_train, epochs=3000, batch_size=3000, callbacks=callbacks_list)

Epoch 1/3000
52945/52945 [==============================] - 96s 2ms/step - loss: 0.6707 - acc: 0.5912

Epoch 00001: acc improved from -inf to 0.59118, saving model to 01-0.5912.hdf5
Epoch 2/3000
52945/52945 [==============================] - 87s 2ms/step - loss: 0.6571 - acc: 0.6008

Epoch 00002: acc improved from 0.59118 to 0.60085, saving model to 02-0.6008.hdf5
Epoch 3/3000
52945/52945 [==============================] - 97s 2ms/step - loss: 0.6404 - acc: 0.6339

Epoch 00003: acc improved from 0.60085 to 0.63387, saving model to 03-0.6339.hdf5
Epoch 4/3000
52945/52945 [==============================] - 99s 2ms/step - loss: 0.6332 - acc: 0.6400

Epoch 00004: acc improved from 0.63387 to 0.64002, saving model to 04-0.6400.hdf5
Epoch 5/3000
52945/52945 [==============================] - 100s 2ms/step - loss: 0.6273 - acc: 0.6423

Epoch 00005: acc improved from 0.64002 to 0.64233, saving model to 05-0.6423.hdf5
Epoch 6/3000
52945/52945 [==============================] - 121s 2ms/step - l

52945/52945 [==============================] - 80s 2ms/step - loss: 0.0542 - acc: 0.9758

Epoch 00046: acc did not improve
Epoch 47/3000
52945/52945 [==============================] - 81s 2ms/step - loss: 0.0538 - acc: 0.9757

Epoch 00047: acc did not improve
Epoch 48/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0519 - acc: 0.9768

Epoch 00048: acc improved from 0.97616 to 0.97677, saving model to 48-0.9768.hdf5
Epoch 49/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0514 - acc: 0.9772

Epoch 00049: acc improved from 0.97677 to 0.97722, saving model to 49-0.9772.hdf5
Epoch 50/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0560 - acc: 0.9755

Epoch 00050: acc did not improve
Epoch 51/3000
52945/52945 [==============================] - 81s 2ms/step - loss: 0.0547 - acc: 0.9755

Epoch 00051: acc did not improve
Epoch 52/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0782 - acc

52945/52945 [==============================] - 80s 2ms/step - loss: 0.0255 - acc: 0.9856

Epoch 00099: acc improved from 0.98532 to 0.98555, saving model to 99-0.9856.hdf5
Epoch 100/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0579 - acc: 0.9753

Epoch 00100: acc did not improve
Epoch 101/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0562 - acc: 0.9761

Epoch 00101: acc did not improve
Epoch 102/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0406 - acc: 0.9814

Epoch 00102: acc did not improve
Epoch 103/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0322 - acc: 0.9834

Epoch 00103: acc did not improve
Epoch 104/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0286 - acc: 0.9842

Epoch 00104: acc did not improve
Epoch 105/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0263 - acc: 0.9852

Epoch 00105: acc did not improve


52945/52945 [==============================] - 80s 2ms/step - loss: 0.0220 - acc: 0.9880

Epoch 00154: acc did not improve
Epoch 155/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0254 - acc: 0.9867

Epoch 00155: acc did not improve
Epoch 156/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0263 - acc: 0.9869

Epoch 00156: acc did not improve
Epoch 157/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0243 - acc: 0.9874

Epoch 00157: acc did not improve
Epoch 158/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0217 - acc: 0.9881

Epoch 00158: acc did not improve
Epoch 159/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0203 - acc: 0.9884

Epoch 00159: acc did not improve
Epoch 160/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0201 - acc: 0.9887

Epoch 00160: acc improved from 0.98865 to 0.98867, saving model to 160-0.9887.hdf5

52945/52945 [==============================] - 83s 2ms/step - loss: 0.0194 - acc: 0.9895

Epoch 00208: acc did not improve
Epoch 209/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0197 - acc: 0.9893

Epoch 00209: acc did not improve
Epoch 210/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0186 - acc: 0.9897

Epoch 00210: acc did not improve
Epoch 211/3000
52945/52945 [==============================] - 81s 2ms/step - loss: 0.0182 - acc: 0.9902

Epoch 00211: acc did not improve
Epoch 212/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0183 - acc: 0.9897

Epoch 00212: acc did not improve
Epoch 213/3000
52945/52945 [==============================] - 80s 2ms/step - loss: 0.0176 - acc: 0.9899

Epoch 00213: acc did not improve
Epoch 214/3000
52945/52945 [==============================] - 81s 2ms/step - loss: 0.0175 - acc: 0.9900

Epoch 00214: acc did not improve
Epoch 215/3000
52945/52945 [=====================

18000/52945 [=========>....................] - ETA: 1:13 - loss: 0.0217 - acc: 0.9889